# Wrangling Wav Files in Python

This sections will cover how to deal with wav files in python.

Modules exist already to do a lot of this work, but the prupose here is to demonstrate the principles of creating a wav file from scratch, regardless of the programming langauage used. 

It should also provide a guide for you as an instructor to create a lightweight utility for students which can be deployed easily which is useful when internet connections break or administraytive privleges don't allow for easy installation of additiona software.


## Benefits to making your own library

- agile: you can change it when you need to
- reusable: it should be something simple enough you can pass it on to students to modify as they please.
- flexible: it should be something you can drop into a project for when administrative priveleges, internet connections and a myriad of other IT disasters conspire against you to derail a lesson plan.

## Jupyter Notebook Shortcuts

| Function                   | macOS | Windows      |
| --------------------------: | :----:  | :------------: |
| Run Cells                  | ⌘+⏎   | CTRL+ENTER   |
| Run Cells and Select Below | ⇧+⏎   | SHIFT+ENTER  |
| Run Cells and Insert Below | ⌥+⏎   | ALT+ENTER    |
| Toggle Line Numbers        | ⇧+L   | SHIFT+L      |
| New Cell Above             | a     | a            |
| New Cell Below             | b     | b            |
| Delete Cell                | d d   | d d          |

## Notebooks

IPython widgets provide some nicer ways


In [ ]:
import wave

as always, we import some functions and variables from the math library

In [ ]:
from math import sin, pi

here we can create our sine wav to be written to a file

In [ ]:
fs = 44100.0   # Sampling Rate 
f0 = 440.0     # Fundamental frequency
duration = 1.0 # in seconds

delta = 2.0 * pi * f0 / fs # how much does the phase change between samples

sine_wave = [sin(delta * i) for i in range(int(duration*fs))]

write to a wav file

In [ ]:
file = wave.open('test.wav', 'wb')

In [ ]:
file.setnchannels(1)
file.setsampwidth(2)
file.setframerate(int(fs))

We currently have a list of sample values and before we can write them to a file we need to turn them into a byte-string.

This is typical data wrangling and will be an epected stage whenever writing data to a file.

It can be a little dauntin for it to be one of the first things to introduce to students. Therefore, it is one of the benefits to abstracting the process away in a library. At first they can use the library you provide, but after a while you can invite them to open up the file and explore a little further.

we nee to import the `struct` module which provides the ability to transform from one data type to another.

In [ ]:
import struct

## Wave library

The [wave library](https://docs.python.org/3/library/wave.html) is a standard python library and can be used

The wave library provides some handy functions to deal with parsing the header of a wav file so you don't have to.

In [ ]:
bit_depth = 16
max_amplitude = 2 ** (bit_depth - 1)
byte_data = b''.join([struct.pack('<h', int(sample * max_amplitude)) for sample in sine_wave])

In [ ]:
filename = 'A440-Hz.wav'
wave_file = wave.open(filename, 'wb')
wave_file.setnchannels(1)  # mono
wave_file.setsampwidth(bit_depth // 8)  # 16-bit depth i.e. 2 bytes
wave_file.setframerate(int(fs))


In [ ]:
wave_file.writeframesraw(byte_data)

In [ ]:
wave_file.close()

In [ ]:
import IPython.display as ipd

ipd.Audio(data=sine_wave, rate=fs)

# Reading a wav file

Reading is a lot more simplistic than writing as a lot of decisions have been made for you.

In general, to read a wav file you should expect to deal with three elements

1. opening a file object in read mode
2. reading byte data
3. transforming byte data into floating point format

For python it will be easiest to keep the audio sample format to a list of float type numbers.
This is closest to what you will find in other programming languages.

This assumes you are enforcing wav files with

- 1 channel (mono)
- 16-bit depth

In [ ]:
import wave
import struct

wave_file = wave.open(filename, 'rb')
p = wave_file.getparams()
frames = wave_file.readframes(p.nframes)
audio_samples = [sample[0] / max_amplitude for sample in struct.iter_unpack('<h',frames)]

The `struct` library's `unpack` functions always return a tuple, even if they are only 1 element long, the sort of arbitrary decision that can snipe some students into paralysis as they try to navigate. Another good reason to remove this kind of operation from view.

## An example library

Below is an example of a possible simple library you could provide.

Modify based on what the fpucs of the lessons is. If it is teaching DSP, now might not be the time to punish students for getting the file extension wrong.

If the focus _is_ to teach holistic programming skills, like how to read error, think perhaps of changing the contents of `if not filename.endswith('.wav'):` to throw a helpful error instead.

The library is incredibly limited, but to an extent that is the point.

Should only be a handful of lines, in this case under 30 lines.


Treads some middle ground, enforce parameters like sample rate and bit depth.

Don't be afraid to admit that you have pitched the library incorrectly.

If students are tripping up at the same point, then you can be agile and alter the library accordingly.


To import, all students should have to type is 

```py
from wav_library import *
```

after which they will have access to the `write_wav_file` and `read_wav_file` functions


In [ ]:
# wav_library
#
# To import, all students should have to type is 
#
# ```py
# from wav_library import *
# ```
#
# After which they will have access to the `write_wav_file` and `read_wav_file` functions
#
import struct
import wave
from math import sin, pi

def write_wav_file(float_data, filename, nchannels=1, bit_depth=16, sample_rate=44100):
    
    normalisation = 1.0 / max(abs(float_data))
    
    float_data = [sample * normalisation for sample in float_data]
    
    if not filename.endswith('.wav'):
        filename += '.wav'
        
    with wave.open(filename, 'wb') as wave_file:
        wave_file.setnchannels(nchannels)
        wave_file.setsampwidth(bit_depth // 8)
        wave_file.setframerate(sample_rate)
                
        max_amplitude = 2 ** (bit_depth - 1)
        byte_data = b''.join([struct.pack('<h', int(sample * max_amplitude)) for sample in float_data])
        
        wave_file.writeframesraw(byte_data)

def read_wav_file(filename):

    if not filename.endswith('.wav'):
        filename += '.wav'
        
    with wave.open(filename, 'rb') as wave_file:
        p = wave_file.getparams()
        frames = wave_file.readframes(p.nframes)
        audio_samples = [sample[0] / max_amplitude for sample in struct.iter_unpack('<h',frames)]
        return audio_samples

Limitations with this approach

There are a lot of drawbacks and stumbling blocks which, rather than pretending they don't exist, are worth being aware of.

- guessing file names could cause confusion down the line
- this doens't support stereo
- not transferrable to other languages, this makes heavy use of python list comprehensions. Givene the prevalence of C languages in audio programming, it may be better to follow a standard for loop structure
- messy returns. The read function must return a lot of variables. This could be done as with an object-orientated approach, but there is tradeoff with the complexity that would be removed and put in its place.
- this assumes parameters that would likely change, expecially if students wish to use there own samples.
- the audio is always normalised using `normalisation = 1.0 / max(abs(float_data))`

In [ ]:
import IPython.display as ipd
ipd.Audio(data=read_wav_file(filename), rate=fs)
